In [1]:
import numpy as np
import os
import utils
import time
#from extractDigitFeatures import extractDigitFeatures
from trainModel import trainModel
from evaluateLabels import evaluateLabels
from evaluateModel import evaluateModel

In [2]:
# There are three versions of MNIST dataset
dataTypes = ['digits-normal.mat', 'digits-scaled.mat', 'digits-jitter.mat']

# You have to implement three types of features
featureTypes = ['hog'] #'pixel', 'hog', 

# Accuracy placeholder
accuracy = np.zeros((len(dataTypes), len(featureTypes)))
val_accuracy = np.zeros((len(dataTypes), len(featureTypes)))
trainSet, validationSet, testSet = 1, 2, 3

In [4]:
def hogFeatures(x):
    # Applying Non Linear Mapping
    img = np.sqrt(x)

    # Computing the channel gradient
    r_grad, c_grad = np.empty(img.shape).astype('longdouble'), np.empty(img.shape).astype('longdouble')
    
    r_grad[1:-1,] = img[2:, :] - img[:-2, :] 
    c_grad[:, 1:-1] = img[:, 2:] - img[:, :-2]
    c_grad[:, 0], c_grad[:, -1] = 0, 0
    r_grad[0, :], r_grad[-1, 0] = 0, 0

    img_magnitude = np.sqrt(np.power(r_grad, 2) + np.power(c_grad, 2))
    img_theta = np.rad2deg(np.arctan(c_grad/(r_grad+0.00000001))) % 180
    orientation_bins = 8
    patch_size = 4
    tot_r, tot_c = img.shape
    hog = np.zeros((int(tot_r/patch_size), int(tot_c/patch_size), orientation_bins))
    for j in range(int(tot_c/patch_size)):
        for i in range(int(tot_r/patch_size)):
            # Extract the Current Patch and weight
            curr_patch = extract_relevant_window(img_theta, patch_size, i, j)
            curr_weight = extract_relevant_window(img_magnitude, patch_size, i, j)
            # Applying Histogram calculations
            hog[j][i] = np.histogram(np.ndarray.flatten(curr_patch), weights=np.ndarray.flatten(curr_weight), 
                                     bins=np.linspace(0, 180, num=(orientation_bins+1)))[0]        
    hog_norm = feature_normalization(hog, 'min-max')
    return hog_norm.ravel()

In [3]:
path = os.path.join('../..', 'data', 'digits-normal.mat')
data = utils.loadmat(path)
x = data['x'][:, :, 0]

In [4]:
img = np.sqrt(x)

# Computing the channel gradient
r_grad, c_grad = np.empty(img.shape).astype('longdouble'), np.empty(img.shape).astype('longdouble')

r_grad[1:-1,] = img[2:, :] - img[:-2, :] 
c_grad[:, 1:-1] = img[:, 2:] - img[:, :-2]
c_grad[:, 0], c_grad[:, -1] = 0, 0
r_grad[0, :], r_grad[-1, 0] = 0, 0

img_magnitude = np.sqrt(np.power(r_grad, 2) + np.power(c_grad, 2))
img_theta = np.rad2deg(np.arctan(c_grad/(r_grad+0.00000001))) % 180

In [6]:
img_magnitude

array([[0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.000